In [3]:
import pandas as pd
import requests
import json

In [54]:
# --- Configuration ---
API_KEY = "gmycKK7YxkJ0nWcdaP6X2a8Zd8a+suoKM3f6T/owScQ7UhLtm6BvOW64EK1NOpHm"
BASE_URL = "https://api.collegefootballdata.com/"
TARGET_TEAM = "Notre Dame"
SEASONS = [2024, 2025] # Previous and Current Season

# --- Headers for Authentication ---
headers = {
    "Authorization": f"Bearer {API_KEY}"
}

In [57]:
def fetch_plays_by_year(year):
    """
    Fetches play-by-play data, prints errors, and returns data.
    """
    params = {
        'year': year,
        'team': TARGET_TEAM,
        'seasonType': 'both'
    }

    print(f"Attempting to fetch data for {TARGET_TEAM} - {year} season...")
    
    try:
        response = requests.get(BASE_URL + "games", headers=headers, params=params)
        
        # Check for non-200 status codes (like 400, 401, 404)
        if response.status_code != 200:
            print("--- API ERROR DETECTED ---")
            print(f"Status Code: {response.status_code}")
            
            # The CFBD API often provides a JSON error message
            try:
                error_details = response.json()
                print(f"API Message: {error_details.get('error', 'No specific error message found in JSON.')}")
            except json.JSONDecodeError:
                print(f"Raw Response: {response.text}")
            
            response.raise_for_status() # Raise the HTTPError
        data = response.json()
        print(f"Successfully retrieved {len(data)} plays for {year}.")
        return data

    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return []

In [56]:
# --- Main Execution ---
all_plays_data = []

for year in SEASONS:
    plays = fetch_plays_by_year(year)
    all_plays_data.extend(plays)

# Convert the combined list of dictionaries into a Pandas DataFrame
if all_plays_data:

    df = pd.DataFrame(all_plays_data)
    print("\n--- Data Summary ---")
    print(f"Total plays retrieved for {TARGET_TEAM}: {len(df)}")
else:
    print("\nNo data retrieved. Please check your API key and connection.")

Attempting to fetch data for Notre Dame - 2024 season...
[{"id":401628332,"season":2024,"week":1,"seasonType":"regular","startDate":"2024-08-31T23:30:00.000Z","startTimeTBD":false,"completed":true,"neutralSite":false,"conferenceGame":false,"attendance":null,"venueId":3795,"venue":"Kyle Field","homeId":245,"homeTeam":"Texas A&M","homeClassification":"fbs","homeConference":"SEC","homePoints":13,"homeLineScores":[3,3,0,7],"homePostgameWinProbability":0.08384600052782992,"homePregameElo":1681,"homePostgameElo":1677,"awayId":87,"awayTeam":"Notre Dame","awayClassification":"fbs","awayConference":"FBS Independents","awayPoints":23,"awayLineScores":[3,3,7,10],"awayPostgameWinProbability":0.9161539994721701,"awayPregameElo":1884,"awayPostgameElo":1888,"excitementIndex":6.4724625583,"highlights":"","notes":null},{"id":401628977,"season":2024,"week":2,"seasonType":"regular","startDate":"2024-09-07T19:30:00.000Z","startTimeTBD":false,"completed":true,"neutralSite":false,"conferenceGame":false,"att

In [46]:
df.head()

,id,season,week,seasonType,startDate,startTimeTBD,completed,neutralSite,conferenceGame,attendance,...,awayClassification,awayConference,awayPoints,awayLineScores,awayPostgameWinProbability,awayPregameElo,awayPostgameElo,excitementIndex,highlights,notes
0,401628332,2024,1,regular,2024-08-31T23:30:00.000Z,False,True,False,False,NaN,...,fbs,FBS Independents,23,"[3, 3, 7, 10]",0.916154,1884,1888,6.472463,,None
1,401628977,2024,2,regular,2024-09-07T19:30:00.000Z,False,True,False,False,77622.0,...,fbs,Mid-American,16,"[10, 3, 0, 3]",0.777233,1446,1476,4.910095,,None
2,401628978,2024,3,regular,2024-09-14T19:30:00.000Z,False,True,False,False,61441.0,...,fbs,FBS Independents,66,"[14, 28, 10, 14]",0.999701,1858,1992,2.111292,,None
3,401628979,2024,4,regular,2024-09-21T19:30:00.000Z,False,True,False,False,77622.0,...,fbs,Mid-American,3,"[0, 3, 0, 0]",0.001351,1458,1450,6.081662,,None
4,401628980,2024,5,regular,2024-09-28T19:30:00.000Z,False,True,False,False,77622.0,...,fbs,ACC,24,"[7, 7, 0, 10]",0.140463,1711,1717,7.347803,,None


In [ ]:
df_2024 = pd.DataFrame()
for week_number in range(1, 15):
    plays_url = BASE_URL + "/plays"
    params = {
        "year" : 2024,
        "week" : week_number,
        "seasonType" : "both",
        "classification" : "fbs"
    }
    response = requests.get(plays_url, headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
        week_df = pd.DataFrame(data)
        df_2024 = pd.concat([df_2024, week_df], ignore_index=True)
    else:
        print("--- API ERROR DETECTED ---")
        print(f"Status Code: {response.status_code}")
        
        # The CFBD API often provides a JSON error message
        try:
            error_details = response.json()
            print(f"API Message: {error_details.get('error', 'No specific error message found in JSON.')}")
        except json.JSONDecodeError:
            print(f"Raw Response: {response.text}")
            break

In [77]:
df_2024.to_csv("./regular_season_2024.csv")

In [84]:
df_2024.columns.values

array(['gameId', 'driveId', 'id', 'driveNumber', 'playNumber', 'offense',
       'offenseConference', 'offenseScore', 'defense',
       'defenseConference', 'defenseScore', 'home', 'away', 'period',
       'clock', 'offenseTimeouts', 'defenseTimeouts', 'yardline',
       'yardsToGoal', 'down', 'distance', 'yardsGained', 'scoring',
       'playType', 'playText', 'ppa', 'wallclock'], dtype=object)

In [85]:
# First, let us estimate the success rate of field goals
fg_df = df_2024[df_2024["playType"].str.contains("Field Goal")]
fg_df = fg_df[["offense", "defense", "yardsToGoal", "scoring", "playType", "playText"]]

In [87]:
fg_df[fg_df["offense"] == "Notre Dame"]

,offense,defense,yardsToGoal,scoring,playType,playText
68,Notre Dame,Penn State,23,True,Field Goal Good,Mitch Jeter 41 yd FG GOOD
157,Notre Dame,Penn State,23,True,Field Goal Good,Mitch Jeter 41 yd FG GOOD
201,Notre Dame,Georgia,26,True,Field Goal Good,Mitch Jeter 44 yd FG GOOD
226,Notre Dame,Georgia,30,True,Field Goal Good,Mitch Jeter 48 yd FG GOOD
271,Notre Dame,Georgia,29,True,Field Goal Good,Mitch Jeter 47 yd FG GOOD
5292,Notre Dame,Indiana,31,True,Field Goal Good,Mitch Jeter 49 yd FG GOOD
5392,Notre Dame,Indiana,15,True,Field Goal Good,Mitch Jeter 33 yd FG GOOD
5419,Notre Dame,Indiana,19,False,Blocked Field Goal,Mitch Jeter 37 yd FG BLOCKED blocked by James ...
24927,Notre Dame,Texas A&M,28,True,Field Goal Good,Mitch Jeter 46 yd FG GOOD
24968,Notre Dame,Texas A&M,8,True,Field Goal Good,Mitch Jeter 26 yd FG GOOD
